In [1]:
#  Flow of model
# 1. Data Preprocessing 
# 2. Model.
# 3. Loss/optimization
# 4. Training. 

In [2]:
pip install pytorch-lightning

     |████████████████████████████████| 913 kB 23.6 MB/s 
     |████████████████████████████████| 636 kB 46.3 MB/s 
     |████████████████████████████████| 829 kB 62.9 MB/s 
     |████████████████████████████████| 234 kB 57.4 MB/s 
     |████████████████████████████████| 118 kB 69.6 MB/s 
     |████████████████████████████████| 10.6 MB 54.5 MB/s 
     |████████████████████████████████| 1.3 MB 36.8 MB/s 
     |████████████████████████████████| 294 kB 46.6 MB/s 
     |████████████████████████████████| 142 kB 55.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=5db8331c7fb105e5e5fdc9824a1fd38fce2f9688450f6981a909960f59882408
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninsta

In [3]:
import numpy as np 
import torch 
import matplotlib.pyplot as plt
import sklearn
import torch.autograd
from torch.utils.data import DataLoader, random_split
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import SGD
from torch.optim import Adam
import os
from torchvision import datasets, transforms

if torch.cuda.is_available ():
  device = torch.device("cuda")
  print('GPU IS AVAILABLE :D') # Turn on GPU in collab 

else:
  
  device = torch.device("cpu")
  print('GPU not available ')



GPU not available 


Increased/ Customized data set - Training and Validation data 

In [4]:
# Data preprocessing 
np.random.seed(50)
Inputs = 3*np.random.random(size = (300,4)) # can generate m = 300 samples and n = 4 features 
print(Inputs)
print(Inputs.shape)
#nputs_vec = Inputs.reshape(-1,1) # (1200 , 1) column vector 

[[1.48380494 0.68424931 0.76642177 1.18898973]
 [1.13194529 2.98972269 1.2245916  2.31568198]
 [2.28161006 0.93002805 1.03962361 1.05529445]
 ...
 [2.59049077 0.68359399 2.82602449 1.5580974 ]
 [0.56272659 0.21924999 0.22727439 0.6997265 ]
 [0.97621273 1.96945661 1.68518757 2.96951736]]
(300, 4)


In [5]:
x_torch = torch.from_numpy(Inputs) # tensor
x_torch.requires_grad_(True) # graident for backprop 
x_torch.retain_grad() # leaf tensor is a root of the tree 
print(x_torch.shape)

torch.Size([300, 4])


In [6]:
# returns labels
def l_labels(x):
  return x[0]**4 + (x[1])**3 + torch.sin(x[2]) + x[3]**2

In [7]:
y = torch.zeros(size = (300 , 1)) # samples

for i in range(x_torch.shape[0]):
  y[i] = l_labels(x_torch[i])

print(y)

tensor([[  7.2750],
        [ 34.6682],
        [ 29.8801],
        [ 28.1064],
        [ 21.9779],
        [ 12.5350],
        [ 17.0246],
        [ 26.4298],
        [ 27.8286],
        [ 26.4782],
        [ 38.2007],
        [ 71.3598],
        [ 62.8726],
        [  5.2020],
        [  9.0132],
        [  3.6863],
        [ 82.2729],
        [ 67.5295],
        [ 18.7042],
        [ 45.0779],
        [  6.5456],
        [ 67.3668],
        [ 17.8121],
        [ 10.1087],
        [  8.9203],
        [  5.7506],
        [  5.9007],
        [ 28.1545],
        [  9.8109],
        [ 23.8320],
        [ 10.6152],
        [  7.6838],
        [ 57.8538],
        [ 41.5441],
        [ 61.8318],
        [  6.2136],
        [ 38.5793],
        [ 34.1701],
        [  9.3817],
        [ 29.5905],
        [  2.1003],
        [  5.7299],
        [ 15.3771],
        [ 22.5463],
        [  9.0749],
        [ 28.6798],
        [  1.1032],
        [ 88.4932],
        [  1.1635],
        [ 13.3415],


In [8]:
f_o = torch.zeros(size = (300,4))

for i in range(x_torch.shape[0]):
  f_o[i] = torch.autograd.functional.jacobian(l_labels , x_torch[i])

print(f_o.shape)
  
print(f_o)

torch.Size([300, 4])
tensor([[13.0674,  1.4046,  0.7204,  2.3780],
        [ 5.8014, 26.8153,  0.3393,  4.6314],
        [47.5099,  2.5949,  0.5065,  2.1106],
        ...,
        [69.5354,  1.4019, -0.9506,  3.1162],
        [ 0.7128,  0.1442,  0.9743,  1.3995],
        [ 3.7213, 11.6363, -0.1141,  5.9390]])


In [9]:
s_o = torch.zeros(size = (300,4,4))

for i in range(x_torch.shape[0]):
  s_o[i] = torch.autograd.functional.hessian(l_labels , x_torch[i])

print(s_o.shape)
  
print(s_o)

torch.Size([300, 4, 4])
tensor([[[26.4201,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  4.1055,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.6936,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  2.0000]],

        [[15.3756,  0.0000,  0.0000,  0.0000],
         [ 0.0000, 17.9383,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.9407,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  2.0000]],

        [[62.4689,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  5.5802,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.8622,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  2.0000]],

        ...,

        [[80.5277,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  4.1016,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.3104,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  2.0000]],

        [[ 3.7999,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.3155,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.2253,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  2.0000]],

        [[11

In [10]:
print(x_torch.shape) # m =300, n =4 so 300 x 4
print(y.shape) # m = 300
print(f_o.shape) # m =300,, n = 4 so 300 x 4
print(s_o.shape) # m = 300, n = 4, n = 4. 

torch.Size([300, 4])
torch.Size([300, 1])
torch.Size([300, 4])
torch.Size([300, 4, 4])


In [11]:
BATCH_SIZE = 25

In [12]:
# Data Collected and pre-processing 

x = x_torch.type(torch.FloatTensor)
y = y.type(torch.FloatTensor)
f_o = f_o.type(torch.FloatTensor)
s_o = s_o.type(torch.FloatTensor)


# Organise inputs, labels and differential labels in training and validation data set 

# Training to validation split is 5:1 ratio 
# Training data 
x_train = x[:250].to(device) 
#y_train = y[:250].to(device)
f_o_train = f_o[:250].to(device)
s_o_train = s_o[:250].to(device)


# Validation Data 
x_valid = x[250:].to(device)
#y_valid = y[250:].to(device)
f_o_valid = f_o[250:].to(device)
s_o_valid = s_o[250:].to(device)


print(x_train.shape)
#print(y_train.shape)
print(f_o_train.shape)
print(s_o_train.shape)
print(x_valid.shape)
#print(y_valid.shape)
print(f_o_valid.shape)
print(s_o_valid.shape)



# trainset_set = torch.utils.data.TensorDataset(x_train, y_train, dy_dx_train)
# validation_set = torch.utils.data.TensorDataset(x_valid, y_valid, dy_dx_valid)

train_dataset = torch.utils.data.TensorDataset(x_train , f_o_train, s_o_train)
validation_dataset = torch.utils.data.TensorDataset(x_valid, f_o_valid, s_o_valid)


train_dataloader = torch.utils.data.DataLoader(train_dataset , batch_size=BATCH_SIZE , shuffle=False , num_workers =0)
val_dataloader = torch.utils.data.DataLoader(validation_dataset , batch_size = BATCH_SIZE, shuffle = False, num_workers = 0 )


torch.Size([250, 4])
torch.Size([250, 4])
torch.Size([250, 4, 4])
torch.Size([50, 4])
torch.Size([50, 4])
torch.Size([50, 4, 4])


In [13]:
input_size = 4 # x features
hidden_size_1 = 36
hidden_size_2 = 36
output_size = 4 # y output of training 

learning_rate = 0.001 # hyperparameter
num_epochs = 500 # hyperparameter    \ 250 ->  1 epoch
                                      # 250/32 = 6.2...    - > updating my weights every 6 training samples. 
#    SGD, ADAM, RMSPROP AND MORE .    Wi+1 = Wi - lr* gradient L / wrt to W   goes to minimum    -> update weights
#                                     Bi+1 = Bi - lr* gradient L / wrt to B  goes to minimum     -> Update biases 

beta = 1
delta = 1

In [ ]:
# class NeuralNet(nn.Module):
#   def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size): 
#     super(NeuralNet, self).__init__()
#     self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
#     self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
#     self.layer_3 = nn.Linear(hidden_size_2, output_size)

# # 3 layer MLP forward pass
#   def forward(self, x):
#     # layer 1
#     x = self.layer_1(x)
#     x = torch.Softplus(x)

#     # layer 2
#     x = self.layer_2(x)
#     x = torch.Softplus(x)

#     # layer 3 
#     x = self.layer_3(x)

#     return  x 




Twin_second_Order Model 

In [14]:
class TwinSecondOrderNetwork(pl.LightningModule):
  
  def __init__(self):
    super(TwinSecondOrderNetwork,self).__init__()
    self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
    self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
    self.layer_3 = nn.Linear(hidden_size_2, output_size)

    self.softplus = nn.Softplus()
    self.loss = nn.MSELoss()

    self.automatic_optimization = False # Turn off automatic optimization. Turns off backprop automatically. 

  def forward(self, x):
    # layer 1
    x = self.layer_1(x)
    x = self.softplus(x)

    # layer 2
    x = self.layer_2(x)
    x = self.softplus(x)

    # layer 3 
    x = self.layer_3(x)

    return x


  def training_step(self, train_batch, batch_idx):
    opt = self.optimizers()
    opt.zero_grad()

    x, f_o, s_o = train_batch
    predicted_f_o = self.forward(x)
    # loop over batch
    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4))
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_s_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True) # https://github.com/pytorch/pytorch/issues/46918 - Can show we need create_graph to be true as it backpropagets through the graph

    loss = beta * self.loss( f_o, predicted_f_o) + delta * self.loss(s_o,  predicted_s_o)

    loss.backward(retain_graph=True) # keeps gradeints the same and not zero. 

    opt.step() # Every batch. 

    self.log('train_loss', loss)
    print(loss)
    return loss
    


  def validation_step(self, val_batch, batch_idx):
    x, f_o, s_o = val_batch
    predicted_f_o = self.forward(x)
    # loop over batch
    predicted_s_o = torch.zeros(size=(x.shape[0], 4 ,4))
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_s_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True)

    loss = beta * self.loss( f_o, predicted_f_o) + delta * self.loss(s_o,  predicted_s_o)
    self.log('val_loss', loss)
   # print( loss)
    return loss


  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-3)
   





In [15]:
trainer = pl.Trainer(max_epochs= 50)
# https://www.machinecurve.com/index.php/question/how-to-set-number-of-epochs-in-pytorch-lightning/
model = TwinSecondOrderNetwork() # Trained. 

trainer.fit(model, train_dataloader, val_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name     | Type     | Params
--------------------------------------
0 | layer_1  | Linear   | 180   
1 | layer_2  | Linear   | 1.3 K 
2 | layer_3  | Linear   | 148   
3 | softplus | Softplus | 0     
4 | loss     | MSELoss  | 0     
--------------------------------------
1.7 K     Trainable params
0         Non-trainable params
1.7 K     Total params
0.007     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


tensor(691.7720, grad_fn=<AddBackward0>)
tensor(455.2380, grad_fn=<AddBackward0>)
tensor(311.4327, grad_fn=<AddBackward0>)
tensor(497.8914, grad_fn=<AddBackward0>)
tensor(450.0844, grad_fn=<AddBackward0>)
tensor(758.4471, grad_fn=<AddBackward0>)
tensor(480.8900, grad_fn=<AddBackward0>)
tensor(199.5171, grad_fn=<AddBackward0>)
tensor(738.4958, grad_fn=<AddBackward0>)
tensor(614.6758, grad_fn=<AddBackward0>)


tensor(680.9463, grad_fn=<AddBackward0>)
tensor(447.7267, grad_fn=<AddBackward0>)
tensor(304.5118, grad_fn=<AddBackward0>)
tensor(489.3120, grad_fn=<AddBackward0>)
tensor(442.7922, grad_fn=<AddBackward0>)
tensor(746.2365, grad_fn=<AddBackward0>)
tensor(471.6473, grad_fn=<AddBackward0>)
tensor(194.2835, grad_fn=<AddBackward0>)
tensor(727.7947, grad_fn=<AddBackward0>)
tensor(603.6330, grad_fn=<AddBackward0>)


tensor(669.2836, grad_fn=<AddBackward0>)
tensor(439.7059, grad_fn=<AddBackward0>)
tensor(297.0364, grad_fn=<AddBackward0>)
tensor(479.9294, grad_fn=<AddBackward0>)
tensor(434.7564, grad_fn=<AddBackward0>)
tensor(732.4019, grad_fn=<AddBackward0>)
tensor(461.1431, grad_fn=<AddBackward0>)
tensor(188.3793, grad_fn=<AddBackward0>)
tensor(715.4551, grad_fn=<AddBackward0>)
tensor(590.6855, grad_fn=<AddBackward0>)


...

tensor(112.2074, grad_fn=<AddBackward0>)
tensor(88.1782, grad_fn=<AddBackward0>)
tensor(67.5913, grad_fn=<AddBackward0>)
tensor(73.6845, grad_fn=<AddBackward0>)
tensor(88.5338, grad_fn=<AddBackward0>)
tensor(124.4010, grad_fn=<AddBackward0>)
tensor(75.2517, grad_fn=<AddBackward0>)
tensor(55.6223, grad_fn=<AddBackward0>)
tensor(126.0194, grad_fn=<AddBackward0>)
tensor(98.0741, grad_fn=<AddBackward0>)


tensor(107.7931, grad_fn=<AddBackward0>)
tensor(85.7330, grad_fn=<AddBackward0>)
tensor(65.8852, grad_fn=<AddBackward0>)
tensor(70.7578, grad_fn=<AddBackward0>)
tensor(85.1930, grad_fn=<AddBackward0>)
tensor(119.7103, grad_fn=<AddBackward0>)
tensor(72.5980, grad_fn=<AddBackward0>)
tensor(54.5200, grad_fn=<AddBackward0>)
tensor(121.1504, grad_fn=<AddBackward0>)
tensor(94.9678, grad_fn=<AddBackward0>)


tensor(103.9866, grad_fn=<AddBackward0>)
tensor(83.3793, grad_fn=<AddBackward0>)
tensor(64.0686, grad_fn=<AddBackward0>)
tensor(68.1781, grad_fn=<AddBackward0>)
tensor(82.1820, grad_fn=<AddBackward0>)
tensor(115.5768, grad_fn=<AddBackward0>)
tensor(70.1811, grad_fn=<AddBackward0>)
tensor(53.1755, grad_fn=<AddBackward0>)
tensor(116.9578, grad_fn=<AddBackward0>)
tensor(92.0214, grad_fn=<AddBackward0>)


tensor(100.5625, grad_fn=<AddBackward0>)
tensor(81.0022, grad_fn=<AddBackward0>)
tensor(62.1156, grad_fn=<AddBackward0>)
tensor(65.8035, grad_fn=<AddBackward0>)
tensor(79.3738, grad_fn=<AddBackward0>)
tensor(111.8295, grad_fn=<AddBackward0>)
tensor(67.8954, grad_fn=<AddBackward0>)
tensor(51.5988, grad_fn=<AddBackward0>)
tensor(113.2155, grad_fn=<AddBackward0>)
tensor(89.1671, grad_fn=<AddBackward0>)


tensor(97.3737, grad_fn=<AddBackward0>)
tensor(78.5694, grad_fn=<AddBackward0>)
tensor(60.0548, grad_fn=<AddBackward0>)
tensor(63.5532, grad_fn=<AddBackward0>)
tensor(76.6975, grad_fn=<AddBackward0>)
tensor(108.3477, grad_fn=<AddBackward0>)
tensor(65.6864, grad_fn=<AddBackward0>)
tensor(49.8464, grad_fn=<AddBackward0>)
tensor(109.7655, grad_fn=<AddBackward0>)
tensor(86.3780, grad_fn=<AddBackward0>)


In [ ]:
# # Neural Network 
# # Pytroch lighting 
# class model(pl.LightningModule):
#   def __init__(self):
#     super(model, self).__init__() 
#     self.fc1 = nn.linear(input_size, hidden_size_1)
#     self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
#     self.out = nn.Linear(hidden_size_2, output_size)
#     self.lr = 0.001
#     self.loss_1 = nn.MSELoss() # lOSS OF THE LABELS ALONE 
#     self.loss_2 = nn.MSEloss() # LOSS OF FIRST ORDER DIFFERENTIAL LABELS AND LABELS TOGETHER. 

# # 3 layer NEURAL NETWORK
# def forward(self, x):
#   batch_size, _ = x.size() # this is subject to change depends on input data 
#   x = x.view(batch_size, -1)
#   x = F.softplus(self.fc1)
#   x = F.softplus(self.fc2)
#   return self.out(x) 

# def configure_optimizers(self):
#   return Adam(self.paramters(), lr = self.lr) # return adam 

# def training_step(self, train_batch): 
#   x , y = train_batch  
#   prediction_prices = self.forward(x)
#   loss = alpha* self.loss_1(prediction_prices, y) + beta * self.loss_2


# class Data(plt.LightningModule):
#   def prepare_data(self):
#     self.









# # Back prop / losss 


# # Training 
# #validation 

In [ ]:
# class NeuralNet(nn.Module):
#   def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size): 
#     super(NeuralNet, self).__init__()
#     self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
#     self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
#     self.layer_3 = nn.Linear(hidden_size_2, output_size)

# # 3 layer MLP forward pass
#   def forward(self, x):
#     # layer 1
#     x = self.layer_1(x)
#     x = torch.Softplus(x)

#     # layer 2
#     x = self.layer_2(x)
#     x = torch.Softplus(x)

#     # layer 3 
#     x = self.layer_3(x)

#     return  x 